In [10]:
import sys
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import MDAnalysis as mda
import MDAnalysis.transformations as trans
import nglview as nv


import warnings
warnings.filterwarnings(action='once')


if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

try:
    import pipeline
except:
    print("adding code to the pythonpath...")
    code = '/home/anna/Documents/code/python'
    if code not in sys.path:
        sys.path.insert(1, code)
    import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *


/home/anna/Documents/cinnabar/cinnabar/__init__.py


In [11]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm44~lig_ejm45","lig_ejm44~lig_ejm49"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["2fs"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        # try:
        # using the pipeline module for analysis
        analysed_pert = analyse(path_to_dir)
        analysed_pert.set_options(analysis_options)
        analysed_pert.set_options({"try pickle":False})
        avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # analysed_pert.plot_graphs()
        write_analysis_file(analysed_pert, final_results_folder, method=method)
        # except Exception as e:
        #     print(e)
        #     print(f"could not analyse {path_to_dir}")

/home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/GROMACS/2fs/lig_ejm31~lig_ejm45
found GROMACS as engine in work_dir
There are 2 repeats for each the bound and the free for /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/GROMACS/2fs/lig_ejm31~lig_ejm45.these are ['bound_0', 'bound_1'] and ['free_0', 'free_1'].
There are 2 calculated values for each the bound and the free leg for the folders in /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/GROMACS/2fs/lig_ejm31~lig_ejm45.
saving the pmf dictionaries for bound and free as pickles.


ValueError: Could not interpret '  nan kcal/mol' as 'Energy'

In [6]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm31~lig_ejm48","lig_ejm43~lig_ejm46", "lig_ejm46~lig_jmc28", "lig_jmc27~lig_jmc28"]
engine = "AMBER"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["new_mapping"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")

/home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/AMBER/new_mapping/lig_ejm31~lig_ejm45
found AMBER as engine in work_dir
There are 3 repeats for each the bound and the free for /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/AMBER/new_mapping/lig_ejm31~lig_ejm45.these are ['bound_0', 'bound_1', 'bound_2'] and ['free_0', 'free_1', 'free_2'].
MBAR free-energy analysis failed!
Unable to analyse values for 0_free, which is repeat free_0 in /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/AMBER/new_mapping/lig_ejm31~lig_ejm45.
MBAR free-energy analysis failed!
Unable to analyse values for 1_free, which is repeat free_1 in /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/AMBER/new_mapping/lig_ejm31~lig_ejm45.
MBAR free-energy analysis failed!
Unable to analyse values for 2_free, which is repeat free_2 in /home/anna/Documents/benchmark/tyk2_benchmark/outputs_extracted/AMBER/new_mapping/lig_ejm31~lig_ejm45.
Unable to compute values f